In [8]:
import pulp
import pandas as pd

# read data
df = pd.read_excel('~/Documents/omsa/isye6501/hw/hw11/data 15.2/diet.xls')

# split into 2 data frames 
# 1 for daily intake limits
limit_df = df.iloc[-2:, 2:]
# 1 for food nutrition, sort by food name
food_df = df.iloc[:-3].sort_values(by='Foods')

**Problem 1**

In [9]:
# problem 1
prob = pulp.LpProblem('army_nut', pulp.LpMinimize)

# variables
amt_vars = pulp.LpVariable.dicts(name='food amt', indices=food_df['Foods'], lowBound=0)

# objective function
prob += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ]['Price/ Serving'].iloc[0] for food in food_df['Foods']]), 'total cost'

# daily intake constraints
for nut in limit_df.columns[1:]:
    min_nut = limit_df[nut].iloc[0]
    max_nut = limit_df[nut].iloc[1]

    prob += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ][nut].iloc[0] for food in food_df['Foods']]) >= min_nut, f'min_{nut}'
    prob += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ][nut].iloc[0] for food in food_df['Foods']]) <= max_nut, f'max_{nut}'

# solve w/o printing output
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# print non-zero results
for food in amt_vars:
    if amt_vars[food].varValue > 0:
        print(f'{food}: {amt_vars[food].varValue} servings')

Celery, Raw: 52.64371 servings
Frozen Broccoli: 0.25960653 servings
Lettuce,Iceberg,Raw: 63.988506 servings
Oranges: 2.2929389 servings
Poached Eggs: 0.14184397 servings
Popcorn,Air-Popped: 13.869322 servings


Here, you can see that my results match the optimal solution stated in the homework description.

**Problem 2**

In [ ]:
# problem 2
prob2 = pulp.LpProblem('army_nut', pulp.LpMinimize)

# variables
amt_vars = pulp.LpVariable.dicts(name='food amt', indices=food_df['Foods'], lowBound=0)
selected_vars = pulp.LpVariable.dicts(name='selected', indices=food_df['Foods'], cat='Binary')

# objective function
prob2 += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ]['Price/ Serving'].iloc[0] for food in food_df['Foods']]), 'total cost'

# daily intake constraints
for nut in limit_df.columns[1:]:
    min_nut = limit_df[nut].iloc[0]
    max_nut = limit_df[nut].iloc[1]

    prob2 += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ][nut].iloc[0] for food in food_df['Foods']]) >= min_nut, f'min_{nut}'
    prob2 += pulp.lpSum([amt_vars[food] * food_df.loc[food_df['Foods'] == food, ][nut].iloc[0] for food in food_df['Foods']]) <= max_nut, f'max_{nut}'

# constraint a: min 0.1 servings
for food in food_df['Foods']:
    prob2 += amt_vars[food] >= 0.1 * selected_vars[food]
    prob2 += amt_vars[food] <= 1000 * selected_vars[food] # linking constraint to ensure that 'selected' is 1 when 'food amt' > 0

# constraint b: cannot select both celery and broccoli
prob2 += selected_vars['Celery, Raw'] + selected_vars['Frozen Broccoli'] <= 1

# constraint c: at least 3 different types of protein excluding tacos (ambiguous), tofu, and soups
# protein types for constraint c
meat_types = ['Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', 'Pork', 'Hamburger W/Toppings', 'Hotdog, Plain']
poultry_types = ['Roasted Chicken', 'Bologna,Turkey']
fish_types = ['Sardines in Oil', 'White Tuna in Water']
egg_types = ['Poached Eggs', 'Scrambled Eggs']

# vars for constraint c
meat_selected = pulp.LpVariable('meat_selected', cat='Binary')
poultry_selected = pulp.LpVariable('poultry_selected', cat='Binary')
fish_selected = pulp.LpVariable('fish_selected', cat='Binary')
egg_selected = pulp.LpVariable('egg_selected', cat='Binary')

# linking constraints for constraint c
prob2 += meat_selected <= pulp.lpSum([selected_vars[food] for food in meat_types if food in selected_vars])
prob2 += poultry_selected <= pulp.lpSum([selected_vars[food] for food in poultry_types if food in selected_vars])
prob2 += fish_selected <= pulp.lpSum([selected_vars[food] for food in fish_types if food in selected_vars])
prob2 += egg_selected <= pulp.lpSum([selected_vars[food] for food in egg_types if food in selected_vars])

# constraint c
prob2 += meat_selected + poultry_selected + fish_selected + egg_selected >= 3

# solve w/o printing output
prob2.solve(pulp.PULP_CBC_CMD(msg=False))

# print non-zero results
for food in amt_vars:
    if amt_vars[food].varValue > 0:
        print(f'{food}: {amt_vars[food].varValue} servings')

Bologna,Turkey: 0.1 servings
Celery, Raw: 51.078545 servings
Frozen Broccoli: 0.22989332 servings
Kielbasa,Prk: 0.1 servings
Lettuce,Iceberg,Raw: 67.676471 servings
Oranges: 2.3738879 servings
Poached Eggs: 0.12033097 servings
Popcorn,Air-Popped: 13.867193 servings


I interpreted constraint C as requiring at least 1 protein from at least 3 of the 4 stated protein types (meat, poultry, fish, eggs), since the goal is to include a variety of protein sources in the diet. Here, I have 1 poultry (turkey bologna), 1 meat (pork kielbasa), and poached eggs. Since there is at least 0.1 servings of each item and the diet does not include broccoli, constraints A and B are also met.